Recognizing Shapes in Objects with OpenCV
=========================================


Import libraries.

In [1]:
import cv2
import numpy as np
import os

Image filenames.

In [2]:
files = (
    'subway.jpg',
    'breakfast.jpg',
    'dinner.jpg',
    'building.jpg',
)
f = os.path.join('images', files[0])

Define a function for viewing images.

In [3]:
def view_image(i):
    cv2.imshow('view', i)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Read an image from file.

In [4]:
i = cv2.imread(f)
view_image(i)

Inspect image contents

In [5]:
print(i.shape)
print(i[0,0,:]) #Top left pixel - BGR

(640, 427, 3)
[22 24  4]


Computing Image Gradients

Gray-scale

In [6]:
i_gray = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
print(i_gray.shape)
print(i_gray[0,0])
view_image(i_gray)

(640, 427)
18


X gradient

In [7]:
sobelx = cv2.Sobel(i_gray, cv2.CV_64F, 1, 0)
abs_sobelx = np.absolute(sobelx)
view_image(abs_sobelx/np.max(abs_sobelx))

Y gradient

In [8]:
sobely = cv2.Sobel(i_gray, cv2.CV_64F, 0, 1)
abs_sobely = np.absolute(sobely)
view_image(abs_sobely/np.max(abs_sobely))

Magnitude of gradient vector

In [9]:
magnitude = np.sqrt(sobelx**2 + sobely**2)
view_image(magnitude / np.max(magnitude))

Recognizing Lines

Canny edge detection

In [10]:
edges = cv2.Canny(i_gray, 200, 250)
view_image(edges)

Theory: Hough transforms

![title](images2/line_diagram.png)

![title](images2/accumulator1.png)

![title](images2/edge_pixel.png)

![title](images2/accumulator2.png)

![title](images2/accumulator3.png)


Hough transform for lines

In [11]:
lines = cv2.HoughLinesP(
    edges,
    rho=1,
    theta=1. * np.pi/180.0, #degrees to radians
    threshold = 20,         #This is for how many votes each gridcell needs to have for to be considered for a line..
    minLineLength = 30,
    maxLineGap = 5,
)
i_lines = i.copy()
for l in lines:
    x1,y1,x2,y2 = l[0] #gives the end points of line segments
    cv2.line(i_lines, (x1,y1), (x2,y2), (0,0,255), thickness=3)
view_image(i_lines)

Hough transform for circles

In [12]:
circles = cv2.HoughCircles(
    i_gray,
    method=cv2.HOUGH_GRADIENT,
    dp=2, #resolution=2px
    minDist=35,
    param1=150,  #high threshold for canny edge detection
    param2=45,   #low threshold for canny edge detection
    minRadius=15, #targeting the two circles around the subways sign
    maxRadius=25
)
i_circles = i.copy()
for x,y,r in circles[0]:
    cv2.circle(
        i_circles,
        (x,y),
        int(r),
        (0,0,255),
        thickness=3
    )
view_image(i_circles)

<ipython-input-12-948159b56d4d>:13: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv2.circle(


Blur the image first

In [13]:
i_blurred = cv2.GaussianBlur(
    i_gray,
    ksize=(21,21),
    sigmaX=0,
)
view_image(i_blurred)

Circle detection on blurred image

In [14]:
circles = cv2.HoughCircles(
    i_blurred,
    method=cv2.HOUGH_GRADIENT,
    dp=2, #resolution=2px
    minDist=35,
    param1=150,  #high threshold for canny edge detection
    param2=45,   #low threshold for canny edge detection
    minRadius=15, #targeting the two circles around the subways sign
    maxRadius=25
)
i_circles = i.copy()
for x,y,r in circles[0]:
    cv2.circle(
        i_circles,
        (x,y),
        int(r),
        (0,0,255),
        thickness=3
    )
view_image(i_circles)

<ipython-input-14-189e93254357>:13: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv2.circle(
